### Import and install required packages and libraries

In [1]:
# uncomment the applicable line to install (needed only once)

# For MacOs and Linux
# %pip install tk
# %pip install openpyxl
# %pip install pandas
# %pip install requests

# For Windows

In [2]:
# import required libraries
import warnings
warnings.filterwarnings("ignore")
import tkinter
import csv
from csv import reader
import os
import json
from tkinter import filedialog
from tkinter import *
import pandas as pd
import requests
from zipfile import ZipFile
from pathlib import Path
import requests

### Select working directory

In [3]:
def browse_button():
    # select a directory and store it in global var "folder_path"
    global folder_path
    filename = filedialog.askdirectory(initialdir='/Users/angel/Dropbox (Last Mile Team)/Mac/Desktop')
    folder_path.set(filename)
    print(filename)
    root.destroy()

root = Tk()
folder_path = StringVar()
lbl1 = Label(master=root,textvariable=folder_path)
lbl1.grid(row=0, column=1)
button2 = Button(text="Browse", command=browse_button)
button2.grid(row=0, column=3)

mainloop()
wdir = os.path.join(folder_path.get())
os.chdir(wdir)

/Users/angel/Dropbox (Last Mile Team)/Lis/22_Madrid LL/LL_2_to-be_lmt_diesel_van _electric_scooter_runs/20220503_to-be_lmt_diesel_van _electric_scooter


### Request energy generation data to REData API

In [4]:
url1 = "https://apidatos.ree.es/en/datos/balance/balance-electrico?start_date={}T00:00&end_date={}T23:59&time_trunc=day"
date1 = wdir.split("/")[-1][:8]
date2 = date1[:4] + "-" + date1[4:6] + "-" + date1[6:8]
r = requests.get(url1.format(date2, date2)).json()
ren = r["included"][0]["attributes"]["content"][-1]["attributes"]["total"]
nren = r["included"][1]["attributes"]["content"][-1]["attributes"]["total"]
tenergy = ren + nren

Combined_cycle = r["included"][1]["attributes"]["content"][2]["attributes"]["total"]
Coal = r["included"][1]["attributes"]["content"][3]["attributes"]["total"]
Diesel_engines = r["included"][1]["attributes"]["content"][4]["attributes"]["total"]
Gas_turbine = r["included"][1]["attributes"]["content"][5]["attributes"]["total"]
Steam_turbine = r["included"][1]["attributes"]["content"][6]["attributes"]["total"] 
Cogeneration = r["included"][1]["attributes"]["content"][7]["attributes"]["total"]
Non_renewable_waste = r["included"][1]["attributes"]["content"][8]["attributes"]["total"]

Combined_cycle_p = 100 * Combined_cycle / tenergy
Coal_p = 100 * Coal/tenergy
Diesel_engines_p = 100 * Diesel_engines/tenergy
Gas_turbine_p = 100 * Gas_turbine / tenergy
Steam_turbine_p = 100 * Steam_turbine/ tenergy
ogeneration_p = 100 * Cogeneration / tenergy
Non_renewable_waste_p = 100 * Non_renewable_waste/ tenergy
df1 = pd.read_excel("lmt_LEAD_input_to_EVCO2_factors.xlsx")
df1["Generation_percentage"] = [Combined_cycle_p, Coal_p, Diesel_engines_p, Gas_turbine_p, Steam_turbine_p, ogeneration_p, Non_renewable_waste_p]
df1.to_excel("lmt_LEAD_input_to_EVCO2_factors.xlsx", index = False)

### Calculate SECOND echelon EVs energy consumption and associated CO2 emissions

In [5]:
input1 = open("lmt_LEAD_input_to_EVCO2_2.json")
lmt_json = json.load(input1)[0]
df2 = pd.read_excel("lmt_LEAD_input_to_EVCO2_2_energy_consumption.xlsx")
a1 = lmt_json["ResponsePlanId"]
a2 = lmt_json["Category"]
a3 = lmt_json["Fuel"]
a4 = lmt_json["Segment"]
a5 = lmt_json["EuroStandard"]
a6 = lmt_json["Stock"]
a7 = lmt_json["MeanActivity"]
#• Energy consumption (TJ)in kWh/100 km for Three wheeler, Electric, Cargo, Vehicle = 8.7
#• Energy consumption (TJ)in kWh/100 km for Transit, Electric, Cargo, Vehicle = 22.7
#• Energy consumption (TJ)in kWh/100 km for Big Transit, Electric, Cargo, Vehicle = 25.0 
if a2 == "Three-wheeler" and a3 == "Electric" and a4 =="Cargo" and a5 == "Vehicle":
    factor = 0.087 
elif a2 == "Transit" and a3 == "Electric" and a4 =="Cargo" and a5 == "Vehicle":
    factor = 0.227
elif a2 == "Big Transit" and a3 == "Electric" and a4 =="Cargo" and a5 == "Vehicle":
    factor = 0.250
else:
    factor = 0.0
a8 = factor * 3.6 * 10**(-6) * a7
a9 = factor * a7

df2_1 = pd.DataFrame([[a1, a2, a3,a4,a5,a6,a7,a8,a9]], columns = df2.columns)
df2_1.to_excel("lmt_LEAD_input_to_EVCO2_2_energy_consumption.xlsx", index = False)

In [6]:
df4 = pd.read_excel("lmt_LEAD_input_to_EVCO2_factors.xlsx")
df4["temp"] = df4["Emissions_Factor"] * df4["Generation_percentage"] / 100
temp = sum(df4["temp"].tolist())
df5 = pd.read_excel("lmt_LEAD_input_to_EVCO2_2_energy_consumption.xlsx")
df5["temp1"] = df5["Stock"] * df5["energykwh"]
temp1 = df5["temp1"].tolist()[0]
if not os.path.exists('./Response_from_EVCO2'):
    os.makedirs('Response_from_EVCO2')


with open('./Response_from_EVCO2/EV_CO2_EMISSIONS_2.csv', "w") as f1:
    print('"","date","emissions_gCO2"', file=f1)
    print('"1",' + date2 + ',' + str(round(temp * temp1, 2)), file=f1)

with open('./Response_from_EVCO2/EV_CO2_EMISSIONS.csv', "w") as f1:
    print('"","date","emissions_gCO2"', file=f1)
    print('"1",' + date2 + ',' + str(round(temp * temp1, 2)), file=f1)

In [ ]:
import shutil

source = 'lmt_LEAD_input_to_EVCO2_2_energy_consumption.xlsx'
target = 'Response_from_EVCO2/lmt_LEAD_input_to_EVCO2_energy_consumption.xlsx'
shutil.copy (source, target)